In [1]:
import pandas as pd 
import datetime
from shapely import wkt, Point
from tqdm import tqdm

In [2]:
tripleg = pd.read_csv('triplegs.csv')
stop_df = pd.read_csv('staypoints.csv')
food_sp_df = pd.read_csv('200m_food_related_stop.csv')

In [3]:
stop_df['started_at'] = pd.to_datetime(stop_df['started_at'])
stop_df['finished_at'] = pd.to_datetime(stop_df['finished_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])
tripleg['started_at'] = pd.to_datetime(tripleg['started_at'])

In [4]:
stop_df['continuous_check'] = ((stop_df['user_id'] == stop_df['user_id'].shift()) & (stop_df['started_at'] == stop_df['finished_at'].shift())).astype(int)

In [5]:
len(stop_df)

432640

In [6]:
ID_list=food_sp_df['id'].unique()
len(ID_list)

51663

In [7]:
def find_trip_before(stay_point_id, time_window = 15):
    time_window = datetime.timedelta(minutes=time_window)
    sp = stop_df[stop_df['id'] == stay_point_id]
    stop_of_user = stop_df[(stop_df['user_id'] == sp.iloc[0]['user_id']) & (stop_df['id'] <= stay_point_id)]
    related_tripled_by_user = tripleg[tripleg['user_id'] == sp.iloc[0]['user_id']]
    
    trip_traveled = []
    stop_point_traveled = []
    # temp_stop_list = []

    i = len(stop_of_user) - 1
    while i >= 0 and stop_of_user.iloc[i]['continuous_check'] == 1:
        stop_point_traveled.insert(0, stop_of_user.iloc[i]['id'])
        # temp_stop_list.insert(0, stop_of_user.iloc[i]['id'])
        i -= 1
    # if len(temp_stop_list) == 0: temp_stop_list.append(stay_point_id)

    while i >= 0:

        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_of_user.iloc[i]['started_at'] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_of_user.iloc[i]['started_at'])]
        
        #try to find trip
        if len(tripled_found) > 0:
            if len(tripled_found) > 1: tripled_found = tripled_found.nlargest(1, 'finished_at')
            trip_traveled.insert(0, tripled_found.iloc[0]['id'])
        #in this case, no trip or stop point were found, end program.
        elif i - 1 >= 0 and stop_of_user.iloc[i]['started_at'] - stop_of_user.iloc[i - 1]['finished_at'] > time_window: 
            stop_point_traveled.insert(0, stop_of_user.iloc[i]['id'])
            break
        
        # if len(temp_stop_list) == 0: temp_stop_list.insert(0, stop_of_user.iloc[i]['id'])
        # temp_stop_list = []
        stop_point_traveled.insert(0, stop_of_user.iloc[i]['id'])

        i -= 1
        if len(tripled_found) > 0 and tripled_found.iloc[0]['started_at'] != stop_of_user.iloc[i]['finished_at']: break

        while i >= 0 and stop_of_user.iloc[i]['continuous_check'] == 1:
            # temp_stop_list.insert(0, stop_of_user.iloc[i]['id'])
            stop_point_traveled.insert(0, stop_of_user.iloc[i]['id'])
            i -= 1
            
    if len(trip_traveled) == 0: return None
    fist_trip = related_tripled_by_user[related_tripled_by_user['id'] == trip_traveled[0]]
    data_to_append = {'deviceID': sp.iloc[0]['user_id'], 'tripleg_ID': trip_traveled, 
                          'trip_start_location': Point(list(wkt.loads(fist_trip.iloc[0]['geom']).coords)[0]),
                          'trip_end_location': sp.iloc[0]['geom'],
                          'trip_start_timestamp': fist_trip.iloc[0]['started_at'],
                          'trip_end_timestamp': sp.iloc[0]['finished_at'],
                          'stop_point_between_trips': stop_point_traveled}

    return data_to_append

In [12]:
#stop_df[(stop_df['id'] >= 418) & (stop_df['id'] <= 418)]

,id,user_id,started_at,finished_at,geom,continuous_check
418,418,22,2022-09-05 00:36:08-04:00,2022-09-05 14:13:11-04:00,POINT (-81.4952983166666627 30.3301469753333315),0


In [9]:
data_to_generate = []
for sp_id in tqdm(ID_list, total=len(ID_list)):
    res = find_trip_before(sp_id)
    if res != None:
        data_to_generate.append(res)

100%|██████████| 51663/51663 [02:19<00:00, 370.35it/s]


In [10]:
output_df = pd.DataFrame.from_records(data_to_generate)
output_df

,deviceID,tripleg_ID,trip_start_location,trip_end_location,trip_start_timestamp,trip_end_timestamp,stop_point_between_trips
0,10,[351],POINT (-81.63582 30.47943),POINT (-81.6282599999999974 30.4753866666666688),2022-09-23 22:00:09-04:00,2022-09-23 23:30:12-04:00,"[292, 293, 294]"
1,10,[400],POINT (-81.64148 30.47423),POINT (-81.6381994300000002 30.4775205850000006),2022-10-07 16:01:14-04:00,2022-10-07 16:48:31-04:00,[335]
2,10,[441],POINT (-81.63404084 30.48468399),POINT (-81.6381949999999961 30.4775149999999968),2022-10-15 21:34:57-04:00,2022-10-15 22:00:32-04:00,"[391, 392, 393]"
3,22,[599],POINT (-81.4048236 30.3449426),POINT (-81.5571999999999946 30.3170449999999967),2022-09-05 16:31:44-04:00,2022-09-05 16:49:22-04:00,"[418, 419, 420, 421, 422, 423, 424]"
4,22,[602],POINT (-81.49559 30.3301),POINT (-81.4864871999999991 30.3180051000000006),2022-09-05 20:50:07-04:00,2022-09-05 21:26:17-04:00,"[428, 429, 430, 431]"
...,...,...,...,...,...,...,...
21725,34203,"[547062, 547063]",POINT (-81.844733 29.042466),POINT (-81.9723630000000014 29.0721200000000017),2022-10-23 12:57:04-04:00,2022-10-23 13:47:34-04:00,"[432305, 432306]"
21726,34405,[547453],POINT (-81.613899 30.2684),POINT (-81.6334250000000026 30.5176349999999985),2022-10-27 19:34:13-04:00,2022-10-27 20:13:21-04:00,"[432539, 432540]"
21727,34471,[547536],POINT (-82.1313439 29.6967879),POINT (-82.0870139999999964 29.5994551000000001),2022-10-20 07:31:35-04:00,2022-10-20 07:46:01-04:00,"[432605, 432606]"
21728,34471,[547538],POINT (-82.18896 29.26703),POINT (-82.2480800000000016 29.4077100000000016),2022-10-22 02:41:05-04:00,2022-10-22 03:00:35-04:00,"[432607, 432608]"


In [ ]:
output_df.to_csv('food_related_tour.csv')